<a href="https://colab.research.google.com/github/e-olang/NLP/blob/main/OpenAi/2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip3 install pytube -q         # any other alt, wiill work, chosse what works best for you
!pip install git+https://github.com/openai/whisper.git -q
!pip install gradio -q

     |████████████████████████████████| 56 kB 4.1 MB/s 
     |████████████████████████████████| 4.9 MB 27.8 MB/s 
     |████████████████████████████████| 6.6 MB 54.2 MB/s 
     |████████████████████████████████| 120 kB 66.3 MB/s 
     |████████████████████████████████| 5.3 MB 30.9 MB/s 
     |████████████████████████████████| 112 kB 69.2 MB/s 
     |████████████████████████████████| 2.3 MB 59.6 MB/s 
     |████████████████████████████████| 212 kB 68.8 MB/s 
     |████████████████████████████████| 57 kB 5.1 MB/s 
     |████████████████████████████████| 54 kB 3.1 MB/s 
     |████████████████████████████████| 84 kB 3.7 MB/s 
     |████████████████████████████████| 84 kB 3.3 MB/s 
     |████████████████████████████████| 270 kB 68.2 MB/s 
     |████████████████████████████████| 55 kB 3.6 MB/s 
     |████████████████████████████████| 63 kB 2.0 MB/s 
     |████████████████████████████████| 80 kB 8.8 MB/s 
     |████████████████████████████████| 68 kB 5.4 MB/s 
     |██████████████████████████

In [ ]:
import io
import os
import numpy as np

try:
    import tensorflow  # required in Colab to avoid protobuf compatibility issues
except ImportError:
    pass

import torch
import pandas as pd
import urllib
import tarfile
import whisper
import torchaudio

from scipy.io import wavfile
from tqdm.notebook import tqdm


pd.options.display.max_rows = 100
pd.options.display.max_colwidth = 1000
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


Moving 0 files to the new cache system


0it [00:00, ?it/s]

In [ ]:
import ipywidgets as widgets

languages = {"af_za": "Afrikaans", "am_et": "Amharic", "ar_eg": "Arabic", "as_in": "Assamese", "az_az": "Azerbaijani", "be_by": "Belarusian", "bg_bg": "Bulgarian", "bn_in": "Bengali", "bs_ba": "Bosnian", "ca_es": "Catalan", "cmn_hans_cn": "Chinese", "cs_cz": "Czech", "cy_gb": "Welsh", "da_dk": "Danish", "de_de": "German", "el_gr": "Greek", "en_us": "English", "es_419": "Spanish", "et_ee": "Estonian", "fa_ir": "Persian", "fi_fi": "Finnish", "fil_ph": "Tagalog", "fr_fr": "French", "gl_es": "Galician", "gu_in": "Gujarati", "ha_ng": "Hausa", "he_il": "Hebrew", "hi_in": "Hindi", "hr_hr": "Croatian", "hu_hu": "Hungarian", "hy_am": "Armenian", "id_id": "Indonesian", "is_is": "Icelandic", "it_it": "Italian", "ja_jp": "Japanese", "jv_id": "Javanese", "ka_ge": "Georgian", "kk_kz": "Kazakh", "km_kh": "Khmer", "kn_in": "Kannada", "ko_kr": "Korean", "lb_lu": "Luxembourgish", "ln_cd": "Lingala", "lo_la": "Lao", "lt_lt": "Lithuanian", "lv_lv": "Latvian", "mi_nz": "Maori", "mk_mk": "Macedonian", "ml_in": "Malayalam", "mn_mn": "Mongolian", "mr_in": "Marathi", "ms_my": "Malay", "mt_mt": "Maltese", "my_mm": "Myanmar", "nb_no": "Norwegian", "ne_np": "Nepali", "nl_nl": "Dutch", "oc_fr": "Occitan", "pa_in": "Punjabi", "pl_pl": "Polish", "ps_af": "Pashto", "pt_br": "Portuguese", "ro_ro": "Romanian", "ru_ru": "Russian", "sd_in": "Sindhi", "sk_sk": "Slovak", "sl_si": "Slovenian", "sn_zw": "Shona", "so_so": "Somali", "sr_rs": "Serbian", "sv_se": "Swedish", "sw_ke": "Swahili", "ta_in": "Tamil", "te_in": "Telugu", "tg_tj": "Tajik", "th_th": "Thai", "tr_tr": "Turkish", "uk_ua": "Ukrainian", "ur_pk": "Urdu", "uz_uz": "Uzbek", "vi_vn": "Vietnamese", "yo_ng": "Yoruba"}
selection = widgets.Dropdown(
    options=[("Select language", None), ("----------", None)] + sorted([(f"{v} ({k})", k) for k, v in languages.items()]),
    value="ko_kr",
    description='Language:',
    disabled=False,
)

selection

Dropdown(description='Language:', index=39, options=(('Select language', None), ('----------', None), ('Afrika…

In [ ]:
lang = selection.value
language = languages[lang]

assert lang is not None, "Please select a language"
print(f"Selected language: {language} ({lang})")

Selected language: Swahili (sw_ke)


In [ ]:
class Fleurs(torch.utils.data.Dataset):
    """
    A simple class to wrap Fleurs and subsample a portion of the dataset as needed.
    """
    def __init__(self, lang, split="test", subsample_rate=1, device=DEVICE):
        url = f"https://storage.googleapis.com/xtreme_translations/FLEURS102/{lang}.tar.gz"
        tar_path = os.path.expanduser(f"~/.cache/fleurs/{lang}.tgz")
        os.makedirs(os.path.dirname(tar_path), exist_ok=True)

        if not os.path.exists(tar_path):
            with urllib.request.urlopen(url) as source, open(tar_path, "wb") as output:
                with tqdm(total=int(source.info().get("Content-Length")), ncols=80, unit='iB', unit_scale=True, unit_divisor=1024) as loop:
                    while True:
                        buffer = source.read(8192)
                        if not buffer:
                            break

                        output.write(buffer)
                        loop.update(len(buffer))

        labels = {}
        all_audio = {}
        with tarfile.open(tar_path, "r:gz") as tar:
            for member in tar.getmembers():
                name = member.name
                if name.endswith(f"{split}.tsv"):
                    labels = pd.read_table(tar.extractfile(member), names=("id", "file_name", "raw_transcription", "transcription", "_", "num_samples", "gender"))

                if f"/{split}/" in name and name.endswith(".wav"):
                    audio_bytes = tar.extractfile(member).read()
                    all_audio[os.path.basename(name)] = wavfile.read(io.BytesIO(audio_bytes))[1]
                    

        self.labels = labels.to_dict("records")[::subsample_rate]
        self.all_audio = all_audio
        self.device = device

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, item):
        record = self.labels[item]
        audio = torch.from_numpy(self.all_audio[record["file_name"]].copy())
        text = record["transcription"]
        
        return (audio, text)

In [ ]:
dataset = Fleurs(lang, subsample_rate=10) 

  0%|                                              | 0.00/2.70G [00:00<?, ?iB/s]

In [ ]:
model = whisper.load_model("medium")
print(
    f"Model is {'multilingual' if model.is_multilingual else 'English-only'} "
    f"and has {sum(np.prod(p.shape) for p in model.parameters()):,} parameters."
)

100%|██████████████████████████████████████| 1.42G/1.42G [00:06<00:00, 226MiB/s]


Model is multilingual and has 762,321,920 parameters.


In [ ]:
options = dict(language=language, beam_size=5, best_of=5)
transcribe_options = dict(task="transcribe", **options)
translate_options = dict(task="translate", **options)


In [ ]:
references = []
transcriptions = []
translations = []

for audio, text in tqdm(dataset):
    transcription = model.transcribe(audio, **transcribe_options)["text"]
    translation = model.transcribe(audio, **translate_options)["text"]
    
    transcriptions.append(transcription)
    translations.append(translation)
    references.append(text)

  0%|          | 0/49 [00:00<?, ?it/s]

In [ ]:
data = pd.DataFrame(dict(reference=references, transcription=transcriptions, translation=translations))
data

,reference,transcription,translation
0,nextgen ni mfumo ambao utawala wa shirikisho la anga unadai ungeruhusu ndege kutumia njia fupi na kuokoa mamilioni ya galoni za mafuta kila mwaka na kukata uzalishaji wa kaboni,Next year Nfumo ambao utawala washirikisho na tanga unedai ungeruhusu ndaige kutumia njiafupi na kukuwa mamilioni ya gililoni ya mafuta kila muwa kaka kukata uzalishaji wakaboni,"Next over, where you would like your association to see many more people on this list These would be the cases shared by a number of church members I am going to discuss everything You know of Church이다 fairly easy If you have any extra questions I am going to answer you You can call when you are free I am not really doubting you But I would definitely encourage other organizations as well"
1,mkuu wa polisi chandra shekar solanki alisema mshtakiwa alitokea mahakamani na uso uliofunikwa,Mkuwa polisi Chakandrasheka Solanki alisema mshutakiwa alitokea makamani na uso ulyofunikwa?,Police officer Kandrasheka Solanki said he was arrested on charges of assaulting a woman
2,"tangu mwaka wa 1966 sandabani zimekuwa mbuga za wanyama pori na inakadiriwa kuwa sasa kuna chui 400 wa royal bengal na kulungu takribani 30,000 katika eneo hilo",Tango mwaka walefu moja miyatisa sitini na sita sandabani zimikwa mbuga za wanyama pori Na inakadiriwa kuwa sasa kunachui miyaine warojo bingar na kulungu takriban 133 katika eneo hilo.,"In the year 1196, San Dabani was the capital of the Puri people. It is estimated that there are about 2,000 royal bengal and about 1,000 latin N.E.O.Hilo."
3,nomino zote ikiwemo sie kwa wewe daima huanza kwa herufi kubwa hata katika katikati mwa sentensi,"Luminozoote ikiwemo siye kwawewe, daima huwaza kwa hirufi kubwa, hata katika katikati mwa sentensi.",All the
4,lakini kwa kuwekwa katika tropiki za juu” digrii chache tu kaskazini mwa istiwai utahitaji kukabiliana na joto jingi kila wakati na jua kali wakati anga ni safi nadra zaidi,"Lakini kwa kuwekwa katika tropiki za ju, degree cha cha tu kaskazini mwa, istiwai uta hitaji kukabiliana na joto jingi kila wakati na jua kali wakati anga nisafi na drazaidi.","But if you are in the tropics of degree 3-2, you will need to face a lot of difficulties every time and know the place when it is clear and helpful."
5,hakuweza kunywa dawa zinazohitajika kutuliza maumivu yake kwani zimepigwa marufuku katika michezo,Hakuweza kuno dawa jinazoitadika kutuliza maumizu yake kwa nizi nitigwa marufuku katika amitiaza.,This is where you need to check where you are ma'am.
6,eneo la schengen hata hivyo hufanya kazi kiasi kama nchi moja kuhusiana na jambo hili,Enayo la skrijen hatahivyo ufanya kazi kiasi kama inchi moja uskusiana na jumbo hili,"Off course, you can work as ahost, an MC, kindergarten teacher and even as aulm built."
7,uropa ni bara ambalo ni dogo kiasi lakini lina nchi nyingi huru. katika hali ya kawaida kusafiri kupitia nchi nyingi kungemaanisha kulazimika kufanya maombi ya visa na udhibiti wa pasipoti mara nyingi,uropa nibara ambalo nindobu kiasi lakini lina inchi niingi huru katika hali ya kawaida kusafiri kupitia inchi niingi huru maalisha kulazimika kufanya maumbi avisa na ubibiti wa pasipoti mara niingi,"Europe is still a small country, but it has a lot of people. In order to travel and meet many people, you have to pray and have a lot of passports."
8,watoto huwekwa kwenye huduma ya malezi kwa sababu nyingine nyingi ambazo huanzia kutelekezwa hadi kudhulumiwa na hata unyang’anyi,Watoto huwekwa kwenye huduma ya malizi kwa sababu nyingine nyingi ambazo huwanzia kutakelezua hadi kuthulumiwa na hata unyanga nyi.,Children are affected by the depression because there are so many other reasons for helping the poor.
9,matumizi ya video za kurekodi imesababisha uvumbuzi muhimu katika tafsiri ya sauti ndogo kusongeshwa kwa uso ambao hudumu sekunde ndogo kadhaa,Matumizi ya video ya kurekodi imesababisho wumbuzi muhimu katika tafsiri ya sauti ndogo kusangesho kwa uso ambao wudumu sekunde ndogo kava

In [ ]:
data.to_csv('file1.csv')